# Project 3 - Create a framework for government R&D survey

# Import necessary modules/libraries

In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import re
import json
import pandas as pd
# to avoid warnings to make the document output more readable
# suppress all warnings by using the warnings module and using the function ‘filterwarnings()’.
import warnings
warnings.filterwarnings('ignore')

# Write a function for getting the text data from a website url

In [2]:
def getdata(url):
    r = requests.get(url)
    return r.text

# Write a function for getting all links from one page and store them in a list

First, in this function we will get all “a href” marked links. As mentioned, this could potentially lead to the scraping of other websites you do not want information from. We have to place some restraints on the function.

Second thing is that we also want the href that don’t show the full HTML link but only a relative link and starts with a “/” to be included in the collection of links.

Third, we want to convert the link to a dictionary with the function dict.fromkeys() to prevent saving duplications of the same link and to speed up the link searching process. The result looks like this:

In [3]:
# create empty dict
dict_href_links = {}

# aşağıdaki fonksiyonda ikinci parametreyi linkin kökünü sabit tutmak için koydum
def get_links(website_link,start_link):
    html_data = getdata(website_link)
    soup = BeautifulSoup(html_data, "html.parser")
    list_links = []
    for link in soup.find_all("a", href=True):
        
        # Append to list if new link contains original link
        if str(link["href"]).startswith((str(start_link))):
            list_links.append(link["href"])
            
        # Include all href that do not start with website link but with "/"
        if str(link["href"]).startswith("/"):
            if link["href"] not in dict_href_links:
                print(link["href"])
                dict_href_links[link["href"]] = None
                link_with_www = website_link + link["href"][1:]
                print("adjusted link =", link_with_www)
                list_links.append(link_with_www)
                
    # Convert list of links to dictionary and define keys as the links and the values as "Not-checked"
    dict_links = dict.fromkeys(list_links, "Not-checked")
    return dict_links

# Write a function that loops over all the subpages

We use a for loop to go through the subpages and use tqdm to obtain insight into the number of steps that have been completed and keep track of the remaining time to complete the process.

In [4]:
def get_subpage_links(l):
    for link in tqdm(l):
        # If not crawled through this page start crawling and get links
        if l[link] == "Not-checked":
            # aşağıdaki satırdaki website parametresi bir aşağıdaki komutta verilecek parametredir ve linkin kök hali içindir.
            dict_links_subpages = get_links(link,website) 
            # Change the dictionary value of the link to "Checked"
            l[link] = "Checked"
        else:
            # Create an empty dictionary in case every link is checked
            dict_links_subpages = {}
        # Add new dictionary to old dictionary
        l = {**dict_links_subpages, **l}
    return l

# Import the URL list and Create the loop

Before we start the loop we have to initialize some variables. We save the website we want to scrape in a variable and convert this variable into a single key dictionary that has the value “Not-checked”. We create a counter “counter” to count the number of “Not-checked” links and we create a second counter “counter2” to count the number of iterations. To communicate back to ourselves we create some print statements.

In the below code I used the try except statement. The try except statement can handle exceptions. Exceptions may happen when you run a program. Exceptions are errors that happen during execution of the program. Python won’t tell you about errors like syntax errors (grammar faults), instead it will abruptly stop. An abrupt exit is bad for both the end user and developer. Instead of an emergency halt, you can use a try except statement to properly deal with the problem. An emergency halt will happen if you do not properly handle exceptions. https://pythonbasics.org/try-except/

In [5]:
with open("url_list.txt",encoding='utf-8') as file_in:
    lines = []
    for line in file_in:
        lines.append(line)
website_list = [x.rstrip() for x in lines] # remove line breaks
website_list

['https://ankaraism.saglik.gov.tr/',
 'https://29mayisdh.saglik.gov.tr/',
 'https://akyurtdh.saglik.gov.tr/',
 'https://mamakdh.saglik.gov.tr/',
 'https://beytepedh.saglik.gov.tr/']

In [6]:
# add website WITH slash on end
# website_list=["https://www.kastamonu.bel.tr/v2/","https://www.ilkadim.bel.tr/"]
links={}
# create dictionary of website
for website in website_list:
    dict_links = {website:"Not-checked"}
    counter, counter2 = None, 0
    # I set it to have a maximum of 2 iterations. otherwise it takes a lot of time and usually two iterations are sufficient.
    while counter2<2:
        try:
            counter2 += 1
            dict_links2 = get_subpage_links(dict_links)
            # Count number of non-values and set counter to 0 if there are no values within the dictionary equal to the string "Not-checked"
            # https://stackoverflow.com/questions/48371856/count-the-number-of-occurrences-of-a-certain-value-in-a-dictionary-in-python
            counter = sum(value == "Not-checked" for value in dict_links2.values())
            # Print some statements
            print("")
            print("THIS IS LOOP ITERATION NUMBER", counter2)
            print("LENGTH OF DICTIONARY WITH LINKS =", len(dict_links2))
            print("NUMBER OF 'Not-checked' LINKS = ", counter)
            print("")
            dict_links = dict_links2
        except:
            pass
        
    links.update(dict_links)
    # Save list in json file
    a_file = open("data.json", "w", encoding='utf-8')
    json.dump(links, a_file, ensure_ascii=False, indent=4)
    a_file.close()

  0%|          | 0/306 [00:00<?, ?it/s]

/?_Dil=1
adjusted link = https://ankaraism.saglik.gov.tr/?_Dil=1
/?_Dil=2
adjusted link = https://ankaraism.saglik.gov.tr/?_Dil=2
/siteagaci
adjusted link = https://ankaraism.saglik.gov.tr/siteagaci
/TR-56188/kurumsal.html
adjusted link = https://ankaraism.saglik.gov.tr/TR-56188/kurumsal.html
/TR-56192/il-saglik-mudurumuz.html
adjusted link = https://ankaraism.saglik.gov.tr/TR-56192/il-saglik-mudurumuz.html
/TR-170387/organizasyon-semasi.html
adjusted link = https://ankaraism.saglik.gov.tr/TR-170387/organizasyon-semasi.html
/TR-56190/baskanlarimiz.html
adjusted link = https://ankaraism.saglik.gov.tr/TR-56190/baskanlarimiz.html
/TR-68325/acil-saglik-hizmetleri-baskani.html
adjusted link = https://ankaraism.saglik.gov.tr/TR-68325/acil-saglik-hizmetleri-baskani.html
/TR-101402/bilkent-kamu-hasthizmetleri-baskani.html
adjusted link = https://ankaraism.saglik.gov.tr/TR-101402/bilkent-kamu-hasthizmetleri-baskani.html
/TR-68323/destek-hizmetleri-baskani.html
adjusted link = https://ankaraism.

  0%|          | 1/306 [00:00<02:12,  2.30it/s]

/Eklenti/174052/0/aciktan-atama-basvuru-formudocx.docx
adjusted link = https://ankaraism.saglik.gov.tr/TR-254920/saglik-bakanligi-merkez-ve-tasra-teskilatinda-istihdam-edilmek-uzere-1356-surekli-isci-engelli-personelin-ise-baslama-islemleri-hakkinda-duyuru.htmlEklenti/174052/0/aciktan-atama-basvuru-formudocx.docx
/Eklenti/174053/0/mal--bildirimidoc.doc
adjusted link = https://ankaraism.saglik.gov.tr/TR-254920/saglik-bakanligi-merkez-ve-tasra-teskilatinda-istihdam-edilmek-uzere-1356-surekli-isci-engelli-personelin-ise-baslama-islemleri-hakkinda-duyuru.htmlEklenti/174053/0/mal--bildirimidoc.doc


  1%|          | 3/306 [00:01<02:02,  2.47it/s]

...

 79%|███████▉  | 123/156 [00:27<00:07,  4.71it/s]

/Resim/444216/0/1jpg.png
adjusted link = https://beytepedh.saglik.gov.tr/TR-426040/hastanemizde-coronavirus-egitimi-verildi.htmlResim/444216/0/1jpg.png
/Resim/444219/0/2jpg.png
adjusted link = https://beytepedh.saglik.gov.tr/TR-426040/hastanemizde-coronavirus-egitimi-verildi.htmlResim/444219/0/2jpg.png
/Resim/444222/0/3jpg.png
adjusted link = https://beytepedh.saglik.gov.tr/TR-426040/hastanemizde-coronavirus-egitimi-verildi.htmlResim/444222/0/3jpg.png
/Resim/444221/0/4jpg.png
adjusted link = https://beytepedh.saglik.gov.tr/TR-426040/hastanemizde-coronavirus-egitimi-verildi.htmlResim/444221/0/4jpg.png
/Resim/444218/0/5jpg.png
adjusted link = https://beytepedh.saglik.gov.tr/TR-426040/hastanemizde-coronavirus-egitimi-verildi.htmlResim/444218/0/5jpg.png
/Resim/444220/0/6jpg.png
adjusted link = https://beytepedh.saglik.gov.tr/TR-426040/hastanemizde-coronavirus-egitimi-verildi.htmlResim/444220/0/6jpg.png
/Resim/444217/0/7jpg.png
adjusted link = https://beytepedh.saglik.gov.tr/TR-426040/hasta

 81%|████████  | 126/156 [00:28<00:06,  4.72it/s]

/TR-474279/hastanemiz-bina-turu-ekibi-tarafindan-saglik-bakanligi-saglikta-kalite-standartlari-geregi-hastanemizde-bina-turu-gerceklestirildi.html
adjusted link = https://beytepedh.saglik.gov.tr/TR-246403/haberler.htmlTR-474279/hastanemiz-bina-turu-ekibi-tarafindan-saglik-bakanligi-saglikta-kalite-standartlari-geregi-hastanemizde-bina-turu-gerceklestirildi.html
/TR-570259/hastanemizin-getat-unitesinde-akupunktur-ve-kupa-islemleri-uygulanmaktadir.html
adjusted link = https://beytepedh.saglik.gov.tr/TR-246403/haberler.htmlTR-570259/hastanemizin-getat-unitesinde-akupunktur-ve-kupa-islemleri-uygulanmaktadir.html
/TR-571620/3-aralik-dunya-engelliler-gunu.html
adjusted link = https://beytepedh.saglik.gov.tr/TR-246403/haberler.htmlTR-571620/3-aralik-dunya-engelliler-gunu.html
/TR-644062/9-subat-dunya-sigarayi-birakma-gunu.html
adjusted link = https://beytepedh.saglik.gov.tr/TR-246403/haberler.htmlTR-644062/9-subat-dunya-sigarayi-birakma-gunu.html
/TR-651121/kanserde-erken-teshis-hayat-kurtari

 81%|████████  | 126/156 [00:49<00:11,  2.55it/s]


As you can see the number of not-checked links decreases until it reaches zero and then the script is finished, which is exactly what we want.

source: https://python.plainenglish.io/scraping-the-subpages-on-a-website-ea2d4e3db113

# Import previously produced json data

In [7]:
import json
data = json.load(open('data.json', 'r', encoding='utf-8'))
data

{'https://ankaraism.saglik.gov.tr/TR-254920/saglik-bakanligi-merkez-ve-tasra-teskilatinda-istihdam-edilmek-uzere-1356-surekli-isci-engelli-personelin-ise-baslama-islemleri-hakkinda-duyuru.html': 'Checked',
 'https://ankaraism.saglik.gov.tr/?_Dil=1': 'Not-checked',
 'https://ankaraism.saglik.gov.tr/?_Dil=2': 'Not-checked',
 'https://ankaraism.saglik.gov.tr/siteagaci': 'Not-checked',
 'https://ankaraism.saglik.gov.tr/TR-56188/kurumsal.html': 'Not-checked',
 'https://ankaraism.saglik.gov.tr/TR-56192/il-saglik-mudurumuz.html': 'Not-checked',
 'https://ankaraism.saglik.gov.tr/TR-170387/organizasyon-semasi.html': 'Not-checked',
 'https://ankaraism.saglik.gov.tr/TR-56190/baskanlarimiz.html': 'Not-checked',
 'https://ankaraism.saglik.gov.tr/TR-68325/acil-saglik-hizmetleri-baskani.html': 'Not-checked',
 'https://ankaraism.saglik.gov.tr/TR-101402/bilkent-kamu-hasthizmetleri-baskani.html': 'Not-checked',
 'https://ankaraism.saglik.gov.tr/TR-68323/destek-hizmetleri-baskani.html': 'Not-checked',
 '

# get links from imported json data

In [8]:
links = [t for t in data]
links[0:5]

['https://ankaraism.saglik.gov.tr/TR-254920/saglik-bakanligi-merkez-ve-tasra-teskilatinda-istihdam-edilmek-uzere-1356-surekli-isci-engelli-personelin-ise-baslama-islemleri-hakkinda-duyuru.html',
 'https://ankaraism.saglik.gov.tr/?_Dil=1',
 'https://ankaraism.saglik.gov.tr/?_Dil=2',
 'https://ankaraism.saglik.gov.tr/siteagaci',
 'https://ankaraism.saglik.gov.tr/TR-56188/kurumsal.html']

# Import the keyword list and Scrape the list of words from url's

In [9]:
with open("keyword_list.txt",encoding='utf-8') as file_in2:
    lines2 = []
    for line in file_in2:
        lines2.append(line)
word_list = [x.rstrip() for x in lines2] # remove line breaks
word_list

['Ar-Ge',
 'AR-GE',
 'Araştırma ve Geliştirme',
 'araştırma ve geliştirme',
 'ARAŞTIRMA VE GELİŞTİRME',
 'Arge']

In [10]:
#arge sözcüğünü de eklesem mi, riskli, normal metnin içinde geçebilir
#word_list=['Ar-Ge','AR-GE','Araştırma ve Geliştirme','araştırma ve geliştirme','ARAŞTIRMA VE GELİŞTİRME','Arge']
count=0
d={}
for url in links:
    print("\nWebsite currently being scraped:", url)
    try:
        for word in word_list:
            r = requests.get(url, allow_redirects=False)
            r.encoding='utf-8'
            soup = BeautifulSoup(r.content, 'html.parser')
            results = soup.body.find_all(string=re.compile('.*{0}.*'.format(word)), recursive=True)
            print ('Found the word "{0}" {1} times\n'.format(word, len(results)))
            count+=len(results)
    except:
        pass
    d.update({url: count})
    print ('Found the list {0} times\n'.format(count))
    b_file = open("data1.json", "w", encoding='utf-8')
    json.dump(d, b_file, ensure_ascii=False, indent=4)
    b_file.close()


Website currently being scraped: https://ankaraism.saglik.gov.tr/TR-254920/saglik-bakanligi-merkez-ve-tasra-teskilatinda-istihdam-edilmek-uzere-1356-surekli-isci-engelli-personelin-ise-baslama-islemleri-hakkinda-duyuru.html
Found the word "Ar-Ge" 0 times

Found the word "AR-GE" 0 times

Found the word "Araştırma ve Geliştirme" 0 times

Found the word "araştırma ve geliştirme" 0 times

Found the word "ARAŞTIRMA VE GELİŞTİRME" 0 times

Found the word "Arge" 0 times

Found the list 0 times


Website currently being scraped: https://ankaraism.saglik.gov.tr/?_Dil=1
Found the word "Ar-Ge" 0 times

Found the word "AR-GE" 0 times

Found the word "Araştırma ve Geliştirme" 0 times

Found the word "araştırma ve geliştirme" 0 times

Found the word "ARAŞTIRMA VE GELİŞTİRME" 0 times

Found the word "Arge" 0 times

Found the list 0 times


Website currently being scraped: https://ankaraism.saglik.gov.tr/?_Dil=2
Found the word "Ar-Ge" 0 times

Found the word "AR-GE" 0 times

Found the word "Araştırma

Found the word "Arge" 0 times

Found the list 0 times


Website currently being scraped: https://ankaraism.saglik.gov.tr/TR-102579/destek-hizmetleri-baskan-yardimcilari.html
Found the word "Ar-Ge" 0 times

Found the word "AR-GE" 0 times

Found the word "Araştırma ve Geliştirme" 0 times

Found the word "araştırma ve geliştirme" 0 times

Found the word "ARAŞTIRMA VE GELİŞTİRME" 0 times

Found the word "Arge" 0 times

Found the list 0 times


Website currently being scraped: https://ankaraism.saglik.gov.tr/TR-219005/haydar-orhan.html
Found the word "Ar-Ge" 0 times

Found the word "AR-GE" 0 times

Found the word "Araştırma ve Geliştirme" 0 times

Found the word "araştırma ve geliştirme" 0 times

Found the word "ARAŞTIRMA VE GELİŞTİRME" 0 times

Found the word "Arge" 0 times

Found the list 0 times


Website currently being scraped: https://ankaraism.saglik.gov.tr/TR-102580/mehmet-ela.html
Found the word "Ar-Ge" 0 times

Found the word "AR-GE" 0 times

Found the word "Araştırma ve Geliştirm

Found the word "Arge" 0 times

Found the list 0 times


Website currently being scraped: https://ankaraism.saglik.gov.tr/TR-219412/birimler.html
Found the word "Ar-Ge" 0 times

Found the word "AR-GE" 0 times

Found the word "Araştırma ve Geliştirme" 0 times

Found the word "araştırma ve geliştirme" 0 times

Found the word "ARAŞTIRMA VE GELİŞTİRME" 0 times

Found the word "Arge" 0 times

Found the list 0 times


Website currently being scraped: https://ankaraism.saglik.gov.tr/TR-219472/acil-saglik-hizmetleri-birimi.html
Found the word "Ar-Ge" 0 times

Found the word "AR-GE" 0 times

Found the word "Araştırma ve Geliştirme" 0 times

Found the word "araştırma ve geliştirme" 0 times

Found the word "ARAŞTIRMA VE GELİŞTİRME" 0 times

Found the word "Arge" 0 times

Found the list 0 times


Website currently being scraped: https://ankaraism.saglik.gov.tr/TR-219473/afetlerde-saglik-hizmetler-birimi.html
Found the word "Ar-Ge" 0 times

Found the word "AR-GE" 0 times

Found the word "Araştırma v

Found the word "ARAŞTIRMA VE GELİŞTİRME" 0 times

Found the word "Arge" 0 times

Found the list 0 times


Website currently being scraped: https://ankaraism.saglik.gov.tr/TR-219499/hastane-hizmetleri-birimi.html
Found the word "Ar-Ge" 0 times

Found the word "AR-GE" 0 times

Found the word "Araştırma ve Geliştirme" 0 times

Found the word "araştırma ve geliştirme" 0 times

Found the word "ARAŞTIRMA VE GELİŞTİRME" 0 times

Found the word "Arge" 0 times

Found the list 0 times


Website currently being scraped: https://ankaraism.saglik.gov.tr/TR-219500/istatistik-analiz-ve-raporlama-birimi.html
Found the word "Ar-Ge" 0 times

Found the word "AR-GE" 0 times

Found the word "Araştırma ve Geliştirme" 0 times

Found the word "araştırma ve geliştirme" 0 times

Found the word "ARAŞTIRMA VE GELİŞTİRME" 0 times

Found the word "Arge" 0 times

Found the list 0 times


Website currently being scraped: https://ankaraism.saglik.gov.tr/TR-219501/finansal-analiz-ve-faturalandirma-birimi.html
Found the

Found the word "Ar-Ge" 0 times

Found the word "AR-GE" 0 times

Found the word "Araştırma ve Geliştirme" 0 times

Found the word "araştırma ve geliştirme" 0 times

Found the word "ARAŞTIRMA VE GELİŞTİRME" 0 times

Found the word "Arge" 0 times

Found the list 0 times


Website currently being scraped: https://ankaraism.saglik.gov.tr/TR-156364/asi-programlari-birimi.html
Found the word "Ar-Ge" 0 times

Found the word "AR-GE" 0 times

Found the word "Araştırma ve Geliştirme" 0 times

Found the word "araştırma ve geliştirme" 0 times

Found the word "ARAŞTIRMA VE GELİŞTİRME" 0 times

Found the word "Arge" 0 times

Found the list 0 times


Website currently being scraped: https://ankaraism.saglik.gov.tr/TR-156383/tuberkuloz-birimi.html
Found the word "Ar-Ge" 0 times

Found the word "AR-GE" 0 times

Found the word "Araştırma ve Geliştirme" 0 times

Found the word "araştırma ve geliştirme" 0 times

Found the word "ARAŞTIRMA VE GELİŞTİRME" 0 times

Found the word "Arge" 0 times

Found the list

...

Found the word "Arge" 0 times

Found the list 3 times


Website currently being scraped: https://beytepedh.saglik.gov.tr/TR-404386/uzm-dr-devrim-ozturk-can.html
Found the word "Ar-Ge" 0 times

Found the word "AR-GE" 0 times

Found the word "Araştırma ve Geliştirme" 0 times

Found the word "araştırma ve geliştirme" 0 times

Found the word "ARAŞTIRMA VE GELİŞTİRME" 0 times

Found the word "Arge" 0 times

Found the list 3 times


Website currently being scraped: https://beytepedh.saglik.gov.tr/TR-290579/noroloji.html
Found the word "Ar-Ge" 0 times

Found the word "AR-GE" 0 times

Found the word "Araştırma ve Geliştirme" 0 times

Found the word "araştırma ve geliştirme" 0 times

Found the word "ARAŞTIRMA VE GELİŞTİRME" 0 times

Found the word "Arge" 0 times

Found the list 3 times


Website currently being scraped: https://beytepedh.saglik.gov.tr/TR-298466/uzm-dr-merve-ekici.html
Found the word "Ar-Ge" 0 times

Found the word "AR-GE" 0 times

Found the word "Araştırma ve Geliştirme" 0 time

Found the word "AR-GE" 0 times

Found the word "Araştırma ve Geliştirme" 0 times

Found the word "araştırma ve geliştirme" 0 times

Found the word "ARAŞTIRMA VE GELİŞTİRME" 0 times

Found the word "Arge" 0 times

Found the list 3 times


Website currently being scraped: https://beytepedh.saglik.gov.tr/TR-250275/cerrahi-doktorlarimiz.html
Found the word "Ar-Ge" 0 times

Found the word "AR-GE" 0 times

Found the word "Araştırma ve Geliştirme" 0 times

Found the word "araştırma ve geliştirme" 0 times

Found the word "ARAŞTIRMA VE GELİŞTİRME" 0 times

Found the word "Arge" 0 times

Found the list 3 times


Website currently being scraped: https://beytepedh.saglik.gov.tr/TR-250305/uroloji.html
Found the word "Ar-Ge" 0 times

Found the word "AR-GE" 0 times

Found the word "Araştırma ve Geliştirme" 0 times

Found the word "araştırma ve geliştirme" 0 times

Found the word "ARAŞTIRMA VE GELİŞTİRME" 0 times

Found the word "Arge" 0 times

Found the list 3 times


Website currently being scraped:

Found the word "araştırma ve geliştirme" 0 times

Found the word "ARAŞTIRMA VE GELİŞTİRME" 0 times

Found the word "Arge" 0 times

Found the list 3 times


Website currently being scraped: https://beytepedh.saglik.gov.tr/TR-357274/dr-ahmet-yunus-yucel.html
Found the word "Ar-Ge" 0 times

Found the word "AR-GE" 0 times

Found the word "Araştırma ve Geliştirme" 0 times

Found the word "araştırma ve geliştirme" 0 times

Found the word "ARAŞTIRMA VE GELİŞTİRME" 0 times

Found the word "Arge" 0 times

Found the list 3 times


Website currently being scraped: https://beytepedh.saglik.gov.tr/TR-357275/dr-nartan-demirok.html
Found the word "Ar-Ge" 0 times

Found the word "AR-GE" 0 times

Found the word "Araştırma ve Geliştirme" 0 times

Found the word "araştırma ve geliştirme" 0 times

Found the word "ARAŞTIRMA VE GELİŞTİRME" 0 times

Found the word "Arge" 0 times

Found the list 3 times


Website currently being scraped: https://beytepedh.saglik.gov.tr/TR-350179/drbegum-marsap.html
Found the 

Found the word "Araştırma ve Geliştirme" 0 times

Found the word "araştırma ve geliştirme" 0 times

Found the word "ARAŞTIRMA VE GELİŞTİRME" 0 times

Found the word "Arge" 0 times

Found the list 3 times


Website currently being scraped: https://beytepedh.saglik.gov.tr/TR-404835/ihlal-bildirimi.html
Found the word "Ar-Ge" 0 times

Found the word "AR-GE" 0 times

Found the word "Araştırma ve Geliştirme" 0 times

Found the word "araştırma ve geliştirme" 0 times

Found the word "ARAŞTIRMA VE GELİŞTİRME" 0 times

Found the word "Arge" 0 times

Found the list 3 times


Website currently being scraped: https://beytepedh.saglik.gov.tr/TR-404831/bilgi-guvenligi.html
Found the word "Ar-Ge" 0 times

Found the word "AR-GE" 0 times

Found the word "Araştırma ve Geliştirme" 0 times

Found the word "araştırma ve geliştirme" 0 times

Found the word "ARAŞTIRMA VE GELİŞTİRME" 0 times

Found the word "Arge" 0 times

Found the list 3 times


Website currently being scraped: https://beytepedh.saglik.gov.

Found the word "Arge" 0 times

Found the list 3 times


Website currently being scraped: https://beytepedh.saglik.gov.tr/TR-286161/sgk.html
Found the word "Ar-Ge" 0 times

Found the word "AR-GE" 0 times

Found the word "Araştırma ve Geliştirme" 0 times

Found the word "araştırma ve geliştirme" 0 times

Found the word "ARAŞTIRMA VE GELİŞTİRME" 0 times

Found the word "Arge" 0 times

Found the list 3 times


Website currently being scraped: https://beytepedh.saglik.gov.tr/TR-286170/sabim.html
Found the word "Ar-Ge" 0 times

Found the word "AR-GE" 0 times

Found the word "Araştırma ve Geliştirme" 0 times

Found the word "araştırma ve geliştirme" 0 times

Found the word "ARAŞTIRMA VE GELİŞTİRME" 0 times

Found the word "Arge" 0 times

Found the list 3 times


Website currently being scraped: https://beytepedh.saglik.gov.tr/TR-286530/saglikta-bulusma-noktasi.html
Found the word "Ar-Ge" 0 times

Found the word "AR-GE" 0 times

Found the word "Araştırma ve Geliştirme" 0 times

Found the word 

# Print the url's scraped and cumulative number of found words

In [11]:
data1 = json.load(open('data1.json', 'r', encoding='utf-8'))
data1

{'https://ankaraism.saglik.gov.tr/TR-254920/saglik-bakanligi-merkez-ve-tasra-teskilatinda-istihdam-edilmek-uzere-1356-surekli-isci-engelli-personelin-ise-baslama-islemleri-hakkinda-duyuru.html': 0,
 'https://ankaraism.saglik.gov.tr/?_Dil=1': 0,
 'https://ankaraism.saglik.gov.tr/?_Dil=2': 0,
 'https://ankaraism.saglik.gov.tr/siteagaci': 0,
 'https://ankaraism.saglik.gov.tr/TR-56188/kurumsal.html': 0,
 'https://ankaraism.saglik.gov.tr/TR-56192/il-saglik-mudurumuz.html': 0,
 'https://ankaraism.saglik.gov.tr/TR-170387/organizasyon-semasi.html': 0,
 'https://ankaraism.saglik.gov.tr/TR-56190/baskanlarimiz.html': 0,
 'https://ankaraism.saglik.gov.tr/TR-68325/acil-saglik-hizmetleri-baskani.html': 0,
 'https://ankaraism.saglik.gov.tr/TR-101402/bilkent-kamu-hasthizmetleri-baskani.html': 0,
 'https://ankaraism.saglik.gov.tr/TR-68323/destek-hizmetleri-baskani.html': 0,
 'https://ankaraism.saglik.gov.tr/TR-101405/etlik-kamu-hasthizmetleri-baskani.html': 0,
 'https://ankaraism.saglik.gov.tr/TR-68330

# Show only the links containing keywords with their frequency

In [12]:
a=0
df = pd.DataFrame(columns=['Link', 'Frequency'])
for x, y in data1.items():
    if y>a:
        b=[]
        b.append(x)
        b.append(y-a)
        temporary_df = pd.DataFrame([b], columns=['Link', 'Frequency'])
        df = df.append(temporary_df, ignore_index=True)
        a=y
df.head()

,Link,Frequency
0,https://ankaraism.saglik.gov.tr/TR-241012/sagl...,3


In [13]:
df.to_excel("sil.xlsx")